In [44]:
!gdown https://drive.google.com/uc?id=1wHRVN7l6quMFTG7seVS0pHh7ULrc78ug

'gdown' is not recognized as an internal or external command,
operable program or batch file.


In [45]:
import pandas as pd
from ast import literal_eval
from sklearn.metrics import f1_score

df = pd.read_csv("./subset_genre.csv")

# interpret cols as lists instead of strings
for col in ['unigrams', 'sentences', 'lem_sent']: 
  df[col] = df[col].apply(literal_eval)

# idk where the col comes from 
#df.drop('Unnamed: 0', axis=1, inplace = True)

In [46]:
!pip install transformers

In [47]:
import pandas as pd
import torch
from transformers import BertTokenizer, BertModel
from torch import nn
from torch.optim import Adam
from tqdm import tqdm

In [48]:
tokenizer = BertTokenizer.from_pretrained('bert-base-cased')
labels = {'drama':0,
          'comedy':1,
          'action':2,
          'thriller':3
          }

class Dataset(torch.utils.data.Dataset):

    def __init__(self, df):

        self.labels = [labels[label] for label in df['genre']]
        self.texts = [tokenizer(text, 
                               padding='max_length', max_length = 512, truncation=True,
                                return_tensors="pt") for text in df['text']]

    def classes(self):
        return self.labels

    def __len__(self):
        return len(self.labels)

    def get_batch_labels(self, idx):
        # Fetch a batch of labels
        return np.array(self.labels[idx])

    def get_batch_texts(self, idx):
        # Fetch a batch of inputs
        return self.texts[idx]

    def __getitem__(self, idx):

        batch_texts = self.get_batch_texts(idx)
        batch_y = self.get_batch_labels(idx)

        return batch_texts, batch_y

In [49]:
class BertClassifier(nn.Module):

    def __init__(self, dropout=0.5):

        super(BertClassifier, self).__init__()

        self.bert = BertModel.from_pretrained('bert-base-cased')
        self.dropout = nn.Dropout(dropout)
        self.linear = nn.Linear(768, 4)
        self.relu = nn.ReLU()

    def forward(self, input_id, mask):

        _, pooled_output = self.bert(input_ids= input_id, attention_mask=mask,return_dict=False)
        dropout_output = self.dropout(pooled_output)
        linear_output = self.linear(dropout_output)
        final_layer = self.relu(linear_output)

        return final_layer


In [50]:
def train(model, train_data, val_data, learning_rate, epochs):

    train, val = Dataset(train_data), Dataset(val_data)

    train_dataloader = torch.utils.data.DataLoader(train, batch_size=2, shuffle=True)
    val_dataloader = torch.utils.data.DataLoader(val, batch_size=2)

    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")

    criterion = nn.CrossEntropyLoss()
    optimizer = Adam(model.parameters(), lr= learning_rate)

    if use_cuda:

            model = model.cuda()
            criterion = criterion.cuda()

    for epoch_num in range(epochs):

            total_acc_train = 0
            total_loss_train = 0

            for train_input, train_label in tqdm(train_dataloader):

                train_label = train_label.type(torch.LongTensor)
                train_label = train_label.to(device)
                mask = train_input['attention_mask'].to(device)
                input_id = train_input['input_ids'].squeeze(1).to(device)

                output = model(input_id, mask)
                
                batch_loss = criterion(output, train_label)
                total_loss_train += batch_loss.item()
                
                acc = (output.argmax(dim=1) == train_label).sum().item()
                total_acc_train += acc

                model.zero_grad()
                batch_loss.backward()
                optimizer.step()
            
            total_acc_val = 0
            total_loss_val = 0

            with torch.no_grad():

                for val_input, val_label in val_dataloader:

                    val_label = val_label.type(torch.LongTensor)
                    val_label = val_label.to(device)
                    mask = val_input['attention_mask'].to(device)
                    input_id = val_input['input_ids'].squeeze(1).to(device)

                    output = model(input_id, mask)

                    batch_loss = criterion(output, val_label)
                    total_loss_val += batch_loss.item()
                    
                    acc = (output.argmax(dim=1) == val_label).sum().item()
                    total_acc_val += acc
            
            print(
                f'Epochs: {epoch_num + 1} | Train Loss: {total_loss_train / len(train_data): .3f} | Train Accuracy: {total_acc_train / len(train_data): .3f} | Val Loss: {total_loss_val / len(val_data): .3f} | Val Accuracy: {total_acc_val / len(val_data): .3f}')
                  

In [51]:
def evaluate(model, test_data):

    test = Dataset(test_data)

    test_dataloader = torch.utils.data.DataLoader(test, batch_size=2)

    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")

    if use_cuda:

        model = model.cuda()

    total_acc_test = 0
    test_f1 = 0.0
    batch_cnt = 0

    with torch.no_grad():

        for test_input, test_label in test_dataloader:

              test_label = test_label.to(device)
              mask = test_input['attention_mask'].to(device)
              input_id = test_input['input_ids'].squeeze(1).to(device)

              output = model(input_id, mask)

              acc = (output.argmax(dim=1) == test_label).sum().item()
              total_acc_test += acc

              y_pred = output.argmax(dim=1)
              test_f1 += f1_score(test_label.cpu().detach().numpy(), y_pred.cpu().detach().numpy(), average = 'weighted')
              batch_cnt += 1
    
    print(f'Test F1: {test_f1/batch_cnt}')
    print(f'Test Accuracy: {total_acc_test / len(test_data): .3f}')

In [52]:
df_new = df

In [53]:
# def get_drama_comedy_action_thriller(genres):
#   for gen in genres:
#     if gen=="drama":
#       return gen
#     elif gen=="comedy":
#       return gen
#     elif gen=="action":
#       return gen
#     elif gen=="thriller":
#       return gen
#     else:
#       return 0

# df_new["genre_new"] = df_new.genres.apply(get_drama_comedy_action_thriller)

In [54]:
#df_new2 = df_new[df_new["genre_new"] != 0]

In [55]:
# df_drama = df_new2[df_new2["genre_new"] == "drama"]
# df_drama_subset = df_drama.sample(500)

# df_comedy = df_new2[df_new2["genre_new"] == "comedy"]
# df_comedy_subset = df_comedy.sample(500)

# df_action = df_new2[df_new2["genre_new"] == "action"]
# df_action_subset = df_action.sample(500)

# df_thriller = df_new2[df_new2["genre_new"] == "thriller"]
# df_thriller_subset = df_thriller.sample(500)

In [56]:
# df_concat = pd.concat([df_drama_subset, df_comedy_subset, df_action_subset, df_thriller_subset])
# print(df_concat)

In [57]:
# def flatten_sent(sentences):
#   return [" ".join(sent) for sent in sentences]

# df["lem_sent_join"] = df.lem_sent.apply(flatten_sent)

In [58]:
# def make_string(sentences):
#   return " ".join(sentences)

# df['text'] = df.lem_sent_join.apply(make_string)

In [59]:
import numpy as np

np.random.seed(112)
df_train, df_val, df_test = np.split(df.sample(frac=1, random_state=42), 
                                     [int(.8*len(df)), int(.9*len(df))])

print(len(df_train),len(df_val), len(df_test))

5582 698 698


In [60]:
EPOCHS = 5
model = BertClassifier()
LR = 1e-6
              
train(model, df_train, df_val, LR, EPOCHS)

evaluate(model, df_test)

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
100%|██████████| 2791/2791 [10:26<00:00,  4.46it/s]


Epochs: 1 | Train Loss:  0.662 | Train Accuracy:  0.365 | Val Loss:  0.543 | Val Accuracy:  0.583


100%|██████████| 2791/2791 [10:26<00:00,  4.45it/s]


Epochs: 2 | Train Loss:  0.498 | Train Accuracy:  0.618 | Val Loss:  0.457 | Val Accuracy:  0.650


100%|██████████| 2791/2791 [10:29<00:00,  4.43it/s]


Epochs: 3 | Train Loss:  0.433 | Train Accuracy:  0.677 | Val Loss:  0.423 | Val Accuracy:  0.668


100%|██████████| 2791/2791 [10:36<00:00,  4.38it/s]


Epochs: 4 | Train Loss:  0.374 | Train Accuracy:  0.729 | Val Loss:  0.406 | Val Accuracy:  0.679


100%|██████████| 2791/2791 [10:37<00:00,  4.38it/s]


Epochs: 5 | Train Loss:  0.321 | Train Accuracy:  0.778 | Val Loss:  0.413 | Val Accuracy:  0.692
Test F1: 0.6943648519579753
Test Accuracy:  0.698


In [61]:
evaluate(model, df_test)

Test F1: 0.6929321872015282
Test Accuracy:  0.695
